In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

examples = [
    {
        "movie": "Top Gun",
        "answer": "✈👨‍✈️🔥",
    },
    {
        "movie": "The Godfather",
        "answer": "👪🔫🍕",
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {movie}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie expert, you express movie through three emojis."),
        example_prompt,
        MessagesPlaceholder(variable_name="history"),
        ("human", "What do you know about {movie}?"),
    ]
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"movie": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)
